In [1]:
# Classification model using TF 2.0 to model the usefulness of ultasonic_flow
# Ref: https://stackabuse.com/tensorflow-2-0-solving-classification-and-regression-problems/
# Data files of ultasonic_flow: https://archive.ics.uci.edu/dataset/19/ultasonic_flow+evaluation
# Ref2 Book: Gianultasonic_flowlo Zaccone, Getting Started with TensorFlow-Packt Publishing (2016)
# Chapter 3 on Classifiers ]
import glob
import os
import pandas as pd
import numpy as np
# Ref for matplotlib: https://www.tutorialspoint.com/matplotlib/index.htm
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.models import Model

# Import Data

In [2]:
def read_data(file_name):
    with open(file_name, 'r') as file:
        string = file.read()
        data = []
        rows = string.split('\n')
        for i in rows:
            data.append(i.split('\t'))
        return data


In [3]:
data1 = read_data("./Meter D")
# data2 = read_data("./Meter C")

# Preprocessing Data

In [4]:
from sklearn.preprocessing import RobustScaler

In [5]:
cols = ['Profile factor', 'Symmetry', 'Crossflow',
        'Flow velocity 1', 'Flow velocity 2', 'Flow velocity 3', 'Flow velocity 4',
        'Speed sound 1', 'Speed sound 2', 'Speed sound 3', 'Speed sound 4',
        'Signal strength 1_1', 'Signal strength 1_2' , 'Signal strength 2_1' , 'Signal strength 2_2' , 'Signal strength 3_1' , 'Signal strength 3_2', 'Signal strength 4_1' , 'Signal strength 4_2',
        'Signal quality 1_1', 'Signal quality 1_2' , 'Signal quality 2_1' , 'Signal quality 2_2' , 'Signal quality 3_1' , 'Signal quality 3_2', 'Signal quality 4_1' , 'Signal quality 4_2',
        'Gain 1_1', 'Gain 1_2', 'Gain 2_1', 'Gain 2_2', 'Gain 3_1', 'Gain 3_2', 'Gain 4_1', 'Gain 4_2',
        'Transit time 1_1', 'Transit time 1_2', 'Transit time 2_1', 'Transit time 2_2', 'Transit time 3_1', 'Transit time 3_2', 'Transit time 4_1', 'Transit time 4_2',
        'class']

Convert to dataframe and convert column `class` in a one-hot encoded form

In [20]:
# Import the CSV file to Panda's DataFrame format.
# df1 = pd.DataFrame(data1, columns=cols)
# df2 = pd.DataFrame(data2, columns=cols)
# df = pd.concat([df1, df2], ignore_index=True)

df = pd.DataFrame(data1, columns=cols)
df.dropna(inplace = True)
df = pd.get_dummies(df, columns=['class'], prefix='class')

Drop columns which don't contribute that much to the output prediction

In [21]:
# Drop certain columns
drop_cols = ['Transit time 1_1', 'Transit time 1_2', 'Transit time 2_1', 'Transit time 2_2', 'Transit time 3_1', 'Transit time 3_2', 'Transit time 4_1', 'Transit time 4_2',
        'Signal strength 1_1', 'Signal strength 1_2' , 'Signal strength 2_1' , 'Signal strength 2_2' , 'Signal strength 3_1' , 'Signal strength 3_2', 'Signal strength 4_1' , 'Signal strength 4_2'
             ]
df.drop(columns=drop_cols, inplace=True)

# Separate features and target columns
class_columns = [col for col in df.columns if col.startswith('class')]
feature_columns = [col for col in df.columns if (col not in class_columns) and col not in drop_cols]

Normalize the continuos columns

In [22]:
# Normalize the feature columns
scaler = RobustScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])

df.head()

,Profile factor,Symmetry,Crossflow,Flow velocity 1,Flow velocity 2,Flow velocity 3,Flow velocity 4,Speed sound 1,Speed sound 2,Speed sound 3,...,Gain 2_1,Gain 2_2,Gain 3_1,Gain 3_2,Gain 4_1,Gain 4_2,class_1,class_2,class_3,class_4
0,0.087538,-0.062980,-0.254455,-0.874991,-0.964069,-0.973949,-0.180545,0.287334,0.336721,0.365831,...,0.000000,0.000000,0.0,0.0,0.0,0.0,1,0,0,0
1,-0.037560,-0.402813,-0.042251,-0.508207,-0.608418,-0.610085,0.002792,0.399826,0.460508,0.486537,...,-0.115942,-0.110345,0.0,0.0,0.0,0.0,1,0,0,0
2,-0.117088,0.004812,0.024442,-0.494766,-0.607258,-0.617229,0.003746,0.449531,0.510393,0.537604,...,-0.115942,-0.110345,0.0,0.0,0.0,0.0,1,0,0,0
3,-0.025841,0.356336,-0.246027,-0.508207,-0.603955,-0.626604,-0.004653,0.491389,0.551039,0.580316,...,-0.115942,-0.110345,0.0,0.0,0.0,0.0,1,0,0,0
4,-0.001474,0.185034,0.326474,-0.508401,-0.611096,-0.619908,-0.007945,0.536734,0.600924,0.629526,...,-0.115942,-0.110345,0.0,0.0,0.0,0.0,1,0,0,0


In [23]:
(df.describe())

,Profile factor,Symmetry,Crossflow,Flow velocity 1,Flow velocity 2,Flow velocity 3,Flow velocity 4,Speed sound 1,Speed sound 2,Speed sound 3,...,Gain 2_1,Gain 2_2,Gain 3_1,Gain 3_2,Gain 4_1,Gain 4_2,class_1,class_2,class_3,class_4
count,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,1.800000e+02,180.000000,180.000000,180.000000,180.000000,180.000000,...,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000
mean,2.597108e+00,7.914769e+00,1.247451e+01,-1.336943e-01,-2.292785e-01,-0.224850,-0.022702,0.752479,-0.125784,0.348329,...,3.755717,3.574704,18.673310,18.486243,0.294118,0.294083,0.283333,0.127778,0.305556,0.283333
std,7.390717e+00,1.570499e+02,4.440244e+01,7.602828e-01,7.575564e-01,0.716565,0.505894,3.399379,3.628441,2.494436,...,7.699420,7.328046,38.223136,37.832646,0.452800,0.452821,0.451874,0.334773,0.461927,0.451874
min,-2.916546e+01,-1.909999e+03,-2.151844e+02,-2.602732e+00,-2.483798e+00,-1.864366,-1.104051,-13.566601,-23.625404,-8.739090,...,-0.173913,-0.165517,-1.237113,-1.224490,-0.006682,-0.006682,0.000000,0.000000,0.000000,0.000000
25%,-7.055786e-02,-3.052505e-01,-4.042829e-01,-5.157500e-01,-6.108284e-01,-0.626537,-0.519660,-0.443863,-0.445035,-0.418059,...,-0.115942,-0.110345,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-9.074772e-16,-5.467415e-15,-3.648123e-15,1.526557e-16,-1.426810e-16,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.294421e-01,6.947495e-01,5.957171e-01,4.842500e-01,3.891716e-01,0.373463,0.480340,0.556137,0.554965,0.581941,...,0.884058,0.889655,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
max,3.937576e+01,5.015842e+02,1.521693e+02,1.107651e+00,9.843332e-01,0.968413,0.780564,21.184652,12.904850,9.316620,...,25.507246,24.275862,107.381443,106.285714,1.020045,1.020045,1.000000,1.000000,1.000000,1.000000


Split the dataset to features and target variables

In [24]:
features = df.drop(columns=class_columns)
target = df[class_columns]

# Train Test Split

In [25]:
X = features
y = target

Making sure of the number of cols ans rows in dataset is accounted for 

In [26]:
from sklearn.model_selection import train_test_split

print(X.shape)
print(y.shape)

# The total data in the dataset is 1728, 20% of them is: 346 (no. of test data)
# Size of train data: 1728 - 346 = 1382
# random_state is a seed. It can be any value. If we keep it same on every
# run, then there will be repeatability in the results.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,
                                                    random_state=42)
print('Shape of X_train and X_test:', X_train.shape, X_test.shape)
print('Shape of y_train and y_test:', y_train.shape, y_test.shape)

(180, 27)
(180, 4)
Shape of X_train and X_test: (144, 27) (36, 27)
Shape of y_train and y_test: (144, 4) (36, 4)


Checking datatypes and converting dataset values to `uint8` or `float8` for model training 

In [27]:
X_train_ = X_train.to_numpy()
y_train_ = y_train.to_numpy()
print(X_train_.dtype, X_train_.shape)
print(y_train_.dtype, y_train_.shape)

float64 (144, 27)
uint8 (144, 4)


In [28]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

Model compialation

In [29]:
import tensorflow as tf

# Define the number of nodes in each layer of the network
DENSE1_SIZE = 24
DENSE2_SIZE = 18
DENSE3_SIZE = 12
# DENSE4_SIZE = 8
DROPOUT_RATE = 0.3

input_shape = X.shape[1]
output_shape = y.shape[1]

# Start the model with an Input layer
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(input_shape,)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(DENSE1_SIZE, activation='relu'))
# model.add(tf.keras.layers.Dropout(DROPOUT_RATE))
model.add(tf.keras.layers.Dense(DENSE2_SIZE, activation='relu'))
# model.add(tf.keras.layers.Dropout(DROPOUT_RATE))
model.add(tf.keras.layers.Dense(DENSE3_SIZE, activation='relu'))
# model.add(tf.keras.layers.Dense(DENSE4_SIZE, activation='relu'))
model.add(tf.keras.layers.Dense(output_shape, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

print(input_shape, output_shape)


27 4


In [30]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 27)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                672       
_________________________________________________________________
dense_5 (Dense)              (None, 18)                450       
_________________________________________________________________
dense_6 (Dense)              (None, 12)                228       
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 52        
Total params: 1,402
Trainable params: 1,402
Non-trainable params: 0
_________________________________________________________________
None


Model Tranining

In [34]:
NUM_OF_EPOCHS = 400
BATCH_SIZE = 2

In [35]:
# With epchs 50, the output results where not matching with the expected results
# No need to make those changes because accuracy is achieved with the initial values itself
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE,
                    epochs=NUM_OF_EPOCHS,
                    verbose=1, validation_split=0.2)

Epoch 1/400
58/58 [==============================] - 0s 851us/step - loss: 1.9433e-04 - acc: 1.0000 - val_loss: 2.9531 - val_acc: 0.8621
Epoch 2/400
58/58 [==============================] - 0s 498us/step - loss: 1.8684e-04 - acc: 1.0000 - val_loss: 2.9248 - val_acc: 0.8621
Epoch 3/400
58/58 [==============================] - 0s 423us/step - loss: 1.3101e-04 - acc: 1.0000 - val_loss: 2.9398 - val_acc: 0.8621
Epoch 4/400
58/58 [==============================] - 0s 421us/step - loss: 1.2279e-04 - acc: 1.0000 - val_loss: 2.9848 - val_acc: 0.8621
Epoch 5/400
58/58 [==============================] - 0s 412us/step - loss: 1.3240e-04 - acc: 1.0000 - val_loss: 2.9995 - val_acc: 0.8621
Epoch 6/400
58/58 [==============================] - 0s 426us/step - loss: 1.1944e-04 - acc: 1.0000 - val_loss: 2.9698 - val_acc: 0.8621
Epoch 7/400
58/58 [==============================] - 0s 417us/step - loss: 1.1598e-04 - acc: 1.0000 - val_loss: 2.9774 - val_acc: 0.8621
Epoch 8/400
58/58 [======================

Testing model for accuracy

In [36]:
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

2/2 [==============================] - 0s 1ms/step - loss: 1.3749 - acc: 0.9167
Test Score: 1.3749265670776367
Test Accuracy: 0.9166666865348816


Save model

In [31]:
# Trying out saving the model in h5 file format
# Ref: https://www.tensorflow.org/tutorials/keras/save_and_load
# We have the model object that needs to be saved
# It save text file with Hex numbers in HDF5 format in the current dir
# This model file has a size of 39.52 KB
model.save('ultasonic_flowClassifyModel.h5')

# Conversion of TensorFlow model to TFLite

In [32]:
def representative_dataset():
    for _ in range(100):
      data =  X_test
      yield [data.astype(np.float32)]

print(representative_dataset())

<generator object representative_dataset at 0x1191ff430>


In [33]:
# Converting a tf.Keras model to a TensorFlow Lite model.
# It is preferred to use TFLiteConverter from saved model and then
# Also provide representative dataset to train the converted TFLite model
# Avoid calling the TFLite converter directly from model
#converter = tf.lite.TFLiteConverter.from_keras_model(model)

tf.saved_model.save(model, "saved_ultasonic_flow_seq_model_keras_dir")
converter = tf.lite.TFLiteConverter.from_saved_model("saved_ultasonic_flow_seq_model_keras_dir")

# Though its size is not much, optimizer is used here to check whether it works on ESP32
# if this is chosen, tf.lite.Optimize.OPTIMIZE_FOR_SIZE, the TFLite does not work on ESP32
# Observed that even with Optimize.DEFAULT the TFLite model does not work on ESP32
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: saved_ultasonic_flow_seq_model_keras_dir/assets


2024-11-02 09:10:42.619604: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2024-11-02 09:10:42.835382: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2024-11-02 09:10:42.835394: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
2024-11-02 09:10:42.835396: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored change_concat_input_ranges.
2024-11-02 09:10:42.835715: I tensorflow/cc/saved_model/reader.cc:38] Reading SavedModel from: saved_ultasonic_flow_seq_model_keras_dir
2024-11-02 09:10:42.836370: I tensorflow/cc/saved_model/reader.cc:90] Reading meta graph with tags { serve }
2024-11-02 09:10:42.836375: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: saved_ultasonic_flow_seq_model_keras_dir
2024-11-02 09:10:42.

In [34]:
# Save the model in TFlite format whose size is just 5 KB
# It brings down the size from 49.52 KB to 3.836 KB, 13 times reduction
with open('ultasonic_flowClassifyModel.tflite', 'wb') as f:
  f.write(tflite_model)

In [35]:
# Run the inference on TFLITE model on Python ... here itself first
# Let us now first try to run this tflinte model on Python itself
# Ref: https://www.tensorflow.org/lite/guide/inference
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="ultasonic_flowClassifyModel.tflite")
interpreter.allocate_tensors()

INFO: Initialized TensorFlow Lite runtime.


In [36]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print('input_details:\n', input_details)
print('output_details:\n', output_details)

input_details:
 [{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([ 1, 27], dtype=int32), 'shape_signature': array([-1, 27], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
output_details:
 [{'name': 'StatefulPartitionedCall:0', 'index': 17, 'shape': array([1, 4], dtype=int32), 'shape_signature': array([-1,  4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [37]:
# Test the model on random input data.
input_shape = input_details[0]['shape']
#print(input_shape)
#print(type(X_test))
#print(X_test.iloc[1])
#print(X_test.iloc[0])
input0_data = np.random.random_sample(input_shape)
print(input0_data)
input0_data = np.array(input0_data, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input0_data)

interpreter.invoke()
# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output0_data = interpreter.get_tensor(output_details[0]['index'])
print(output0_data)

# Verify if the same data is given to the original model what is the output
output0_data = model.predict(input0_data)
print(output0_data)

# X_text.iloc[19]: 19th row in ultrasonic flow dataset with output 1.0 0.0 0.0 0.0
input1_data = [[0.42486196756362915,
 0.792097806930542,
 0.5861948728561401,
 0.5610111951828003,
 0.5354955196380615,
 0.43262410163879395,
 0.5789740085601807,
 0.39168402552604675,
 0.6487962603569031,
 0.4895608425140381,
 0.013251636177301407,
 0.8533219695091248,
 0.7751578092575073,
 0.8866511583328247,
 0.7434033155441284,
 0.7136684060096741,
 0.603962779045105,
 0.904298722743988,
 0.8162965178489685,
 0.01068122498691082,
 0.01068122498691082,
 0.006772009190171957,
 0.006772009190171957,
 0.002277904422953725,
 0.002277904422953725,
 0.006507592275738716,
 0.006507592275738716]]
print(input1_data)
input1_data = np.array(input1_data, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input1_data)

interpreter.invoke()
# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output1_data = interpreter.get_tensor(output_details[0]['index'])
print('output1:')
print(output1_data)

# Verify if the same data is given to the original model what is the output
output1_data = model.predict(input1_data)
print(output1_data)

# X_text.iloc[19]: 19th row in ultrasonic flow dataset with output 0.0 0.0 1.0 0.0
input2_data = [[0.4233745038509369,
 0.791832447052002,
 0.5857370495796204,
 0.9354183077812195,
 0.931196928024292,
 0.9218912720680237,
 0.9492834210395813,
 0.39765626192092896,
 0.6546631455421448,
 0.5017998814582825,
 0.01559414528310299,
 0.7993514537811279,
 0.6766496896743774,
 0.7790806889533997,
 0.6904633641242981,
 0.7447614669799805,
 0.6252105236053467,
 0.8376504778862,
 0.7461320757865906,
 0.008544979616999626,
 0.008544979616999626,
 0.0022573363967239857,
 0.0022573363967239857,
 0.01138952188193798,
 0.01138952188193798,
 0.006507592275738716,
 0.006507592275738716]]
print(input2_data)
input2_data = np.array(input2_data, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], tf.Variable(input2_data))

interpreter.invoke()
# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output2_data = interpreter.get_tensor(output_details[0]['index'])
print('output2:')
print(output2_data)

# Verify if the same data is given to the original model what is the output
output2_data = model.predict(input2_data)
print(output2_data)


[[0.10923219 0.86996994 0.13667912 0.04998717 0.0461822  0.92275136
  0.746046   0.63688112 0.2848207  0.3912809  0.02632032 0.155477
  0.79883207 0.86733256 0.63005291 0.30329473 0.90649983 0.68553506
  0.81327096 0.17307036 0.39214352 0.03327899 0.85221482 0.71211587
  0.77083709 0.58047406 0.22967039]]
[[0.99609375 0.         0.         0.        ]]
[[1.0000000e+00 3.4654166e-10 4.1300779e-16 3.3311268e-10]]
[[0.42486196756362915, 0.792097806930542, 0.5861948728561401, 0.5610111951828003, 0.5354955196380615, 0.43262410163879395, 0.5789740085601807, 0.39168402552604675, 0.6487962603569031, 0.4895608425140381, 0.013251636177301407, 0.8533219695091248, 0.7751578092575073, 0.8866511583328247, 0.7434033155441284, 0.7136684060096741, 0.603962779045105, 0.904298722743988, 0.8162965178489685, 0.01068122498691082, 0.01068122498691082, 0.006772009190171957, 0.006772009190171957, 0.002277904422953725, 0.002277904422953725, 0.006507592275738716, 0.006507592275738716]]
output1:
[[0.99609375 0.  

In [38]:
# Function to convert some hex values into an array for C programming
import time, sys

# Function to convert some hex values into an array for C programming
def hex_to_c_array(hex_data, var_name):
    c_str = ""

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += "#define " + var_name.upper() + '_H\n\n'

    c_str += "/*\n Author: Smaran Rangarajan Bharadwaj ft. Mouli Sankaran \n"
    c_str += " CAUTION: This is an auto generated file.\n DO NOT EDIT OR MAKE ANY CHANGES TO IT.\n"

# Time stamping of this model data in the generated file
    localtime = time.asctime( time.localtime(time.time()) )
    c_str += " This model data was generated on " + localtime+ '\n\n'
    print("This model data was generated on:", localtime)

# Add information about the verisons of tools and packages used in generating this header file
    c_str += " Tools used:\n Python:" + str(sys.version) + "\n Numpy:" + str(np.version.version) + \
          "\n TensorFlow:" + str(sys.version) + "\n Keras: "+ str(tf.keras.__version__) + "\n\n"
    print("Tools used: Python:", sys.version, "\n Numpy:", np.version.version, \
          "\n TensorFlow:", sys.version, "\n Keras: ", tf.keras.__version__, "\n\n")

# Training details of the model
    c_str += ' Model details are:\n'
    c_str += ' NUM_OF_EPOCHS = ' + str(NUM_OF_EPOCHS) + '\n'
    c_str += ' BATCH_SIZE    = ' + str(BATCH_SIZE) + '\n*/\n'

# Generate 'C' constants for the no. of nodes in each layer
    c_str += '\nconst int ' + 'DENSE1_SIZE' + ' = ' + str(DENSE1_SIZE) + ';\n'
    c_str +=   'const int ' + 'DENSE2_SIZE' + ' = ' + str(DENSE2_SIZE) + ';\n'
    c_str +=   'const int ' + 'DENSE3_SIZE' + ' = ' + str(DENSE3_SIZE) + ';\n'

    # Add array length at the top of the file
    c_str += '\nconst unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'alignas(8) const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data):
        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formating so each line stays within 80 characters
        if (i + 1) < len(hex_data):
          hex_str += ','
        if (i + 1) % 12 == 0:
          hex_str += '\n'
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n' + format(''.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

In [39]:
# Write TFLite model to a C source (or header) file
with open("ultasonic_flow_model_esp32" + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, "ultasonic_flow_model_esp32"))

This model data was generated on: Sat Nov  2 09:10:45 2024
Tools used: Python: 3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:52:09) 
[Clang 14.0.6 ] 
 Numpy: 1.19.5 
 TensorFlow: 3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:52:09) 
[Clang 14.0.6 ] 
 Keras:  2.6.0 




In [40]:
X_test

,Profile factor,Symmetry,Crossflow,Flow velocity 1,Flow velocity 2,Flow velocity 3,Flow velocity 4,Speed sound 1,Speed sound 2,Speed sound 3,...,Signal quality 4_1,Signal quality 4_2,Gain 1_1,Gain 1_2,Gain 2_1,Gain 2_2,Gain 3_1,Gain 3_2,Gain 4_1,Gain 4_2
19,-0.044899,0.210775,0.156433,-0.521165,-0.626629,-0.638837,-0.012908,0.044910,0.075289,0.100279,...,0.058326,0.083935,0.000000,0.000000,0.000000,0.000000,-0.989691,-0.979592,0.000000,0.000000
42,-0.099521,-0.661037,-0.297637,0.093461,-0.007186,-0.006206,0.309482,-0.421626,-0.423557,-0.389044,...,-0.017669,-0.028332,0.069007,0.069007,0.057971,0.055172,0.247423,0.244898,-0.006682,-0.006682
153,8.013053,49.257046,63.801922,-0.404932,-0.571639,-0.588477,-0.588924,5.500327,2.551963,4.809657,...,-1.083030,-1.149541,6.641907,6.641907,14.202899,13.517241,96.302406,95.319725,1.000000,1.000000
78,-0.016054,-0.144443,-0.243299,-0.514493,-0.608507,-0.619640,-0.001646,0.846305,0.927021,0.958217,...,0.049016,0.061380,-0.086259,-0.086259,-0.115942,-0.110345,0.000000,0.000000,0.000000,0.000000
145,6.332671,33.703640,45.260239,0.678206,0.556910,0.552224,-0.405111,0.555047,0.612933,0.638812,...,-1.070470,-1.114783,-0.051755,-0.051755,-0.173913,-0.165517,-1.237113,-1.224490,1.004454,1.004454
15,0.001474,0.051071,-0.248727,0.483162,0.406311,0.379177,0.487378,-0.382385,-0.381986,-0.351903,...,0.007890,0.004147,-0.017252,-0.017252,-0.115942,-0.110345,0.000000,0.000000,0.000000,0.000000
24,-0.131271,0.081299,-0.628959,-0.514976,-0.617613,-0.640534,-0.001074,0.364944,0.412471,0.448468,...,0.060805,0.080009,0.000000,0.000000,0.000000,0.000000,-0.989691,-0.979592,0.000000,0.000000
68,0.032658,-0.392286,-0.071306,-1.256956,-1.359355,-1.361654,-0.368272,-0.214083,-0.200924,-0.177344,...,0.009038,0.043780,0.099197,0.099197,-0.057971,-0.055172,0.247423,0.244898,-0.004454,-0.004454
113,0.118195,-0.142810,-0.780520,-0.526387,-0.592885,-0.613479,-0.002505,-0.951820,-0.969515,-0.936862,...,0.001693,0.014451,0.000000,0.000000,0.115942,0.110345,0.247423,0.244898,-0.004454,-0.004454
118,0.111187,0.117421,0.376623,-0.235030,-0.318827,-0.326227,0.125477,-0.710268,-0.721016,-0.685237,...,-0.002524,0.005192,0.000000,0.000000,0.115942,0.110345,0.247423,0.244898,-0.004454,-0.004454


In [41]:
X_test.loc[172].tolist()


[-6.152505874633789,
 -4.930311679840088,
 90.71279907226562,
 -1.7821346521377563,
 -1.8033833503723145,
 -1.8469539880752563,
 -0.6111137866973877,
 2.2860257625579834,
 0.4715935289859772,
 -8.710306167602539,
 0.9712690114974976,
 -9.214278221130371,
 -6.4440202713012695,
 -8.791808128356934,
 -11.767769813537598,
 -6.560761451721191,
 -6.589517116546631,
 -0.968554675579071,
 -0.9775245189666748,
 7.901281833648682,
 7.901281833648682,
 25.507246017456055,
 24.275861740112305,
 107.3814468383789,
 106.28571319580078,
 1.0089086294174194,
 1.0089086294174194]

In [42]:
y_test.loc[172]

class_1    0.0
class_2    0.0
class_3    0.0
class_4    1.0
Name: 172, dtype: float32